In [1]:
import findspark

findspark.init()
 

In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext(master="local[2]")

C:\Users\gprak\anaconda3\envs\my36v2\lib\site-packages\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
import pyspark

from pyspark.sql import SparkSession

# spark = SparkSession.builder.getOrCreate()

# df = spark.sql("select 'spark' as hello ")

# df.show()

In [5]:
#sc = spark

In [6]:
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [7]:
spark = SparkSession.builder.appName("TextClassifierApp").getOrCreate()

In [8]:

df = spark.read.csv('../Zycus/86b13d9a4b8e11ec/project/training_data.csv',header=True,inferSchema=True)


In [9]:
df.show()

+--------------------+--------------------+
|               title|            category|
+--------------------+--------------------+
|    The Three Amigos|                None|
|Home Essentials B...|      Home & Kitchen|
|Cooper Wiring Qui...|Tools & Home Impr...|
|Baseboarders&reg;...|Tools & Home Impr...|
|The Great Wave Of...|     Office Products|
|Nemcor Pittsburgh...|      Home & Kitchen|
|Patrician Berkley...|                None|
|SouvNear 81461402...|                None|
|20 Qty. Halco 50W...|Tools & Home Impr...|
|      Rilakkuma Bowl|                None|
|Redbirdlinen 1pc ...|      Home & Kitchen|
|Hospital Bath Tow...|      Home & Kitchen|
|Symphony in Red a...|      Home & Kitchen|
|Big Train BLENDED...|Grocery & Gourmet...|
|Surpahs Round 11 ...|      Home & Kitchen|
|Mikasa Love Story...|                None|
|180 Snacks Nutty ...|                None|
|Anti-Slip Handle ...|      Home & Kitchen|
|Imagine Thicket G...|                None|
|KOHLER K-3754-96 ...|          

In [10]:
df.columns

['title', 'category']

In [11]:
#df=df.dropna()

In [12]:
df.columns

['title', 'category']

In [13]:
import pyspark.ml.feature

In [14]:
dir(pyspark.ml.feature)

['Binarizer',
 'BucketedRandomProjectionLSH',
 'BucketedRandomProjectionLSHModel',
 'Bucketizer',
 'ChiSqSelector',
 'ChiSqSelectorModel',
 'CountVectorizer',
 'CountVectorizerModel',
 'DCT',
 'ElementwiseProduct',
 'FeatureHasher',
 'HasFeaturesCol',
 'HasHandleInvalid',
 'HasInputCol',
 'HasInputCols',
 'HasLabelCol',
 'HasMaxIter',
 'HasNumFeatures',
 'HasOutputCol',
 'HasOutputCols',
 'HasRelativeError',
 'HasSeed',
 'HasStepSize',
 'HasThreshold',
 'HasThresholds',
 'HashingTF',
 'IDF',
 'IDFModel',
 'Imputer',
 'ImputerModel',
 'IndexToString',
 'Interaction',
 'JavaEstimator',
 'JavaMLReadable',
 'JavaMLWritable',
 'JavaModel',
 'JavaParams',
 'JavaTransformer',
 'MaxAbsScaler',
 'MaxAbsScalerModel',
 'MinHashLSH',
 'MinHashLSHModel',
 'MinMaxScaler',
 'MinMaxScalerModel',
 'NGram',
 'Normalizer',
 'OneHotEncoder',
 'OneHotEncoderModel',
 'PCA',
 'PCAModel',
 'Param',
 'Params',
 'PolynomialExpansion',
 'QuantileDiscretizer',
 'RFormula',
 'RFormulaModel',
 'RegexTokenizer',
 'R

In [15]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF

In [16]:
from pyspark.ml.feature import StringIndexer

In [17]:
tokenizer = Tokenizer(inputCol='title',outputCol='mytokens')
stopwords_remover = StopWordsRemover(inputCol='mytokens',outputCol='filtered_tokens')
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures')
idf = IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')

In [18]:
labelEncoder = StringIndexer(inputCol='category',outputCol='label').fit(df)

In [19]:
labelEncoder.transform(df).show(5)

+--------------------+--------------------+-----+
|               title|            category|label|
+--------------------+--------------------+-----+
|    The Three Amigos|                None|  0.0|
|Home Essentials B...|      Home & Kitchen|  1.0|
|Cooper Wiring Qui...|Tools & Home Impr...|  2.0|
|Baseboarders&reg;...|Tools & Home Impr...|  2.0|
|The Great Wave Of...|     Office Products|  3.0|
+--------------------+--------------------+-----+
only showing top 5 rows



In [20]:
label_dict = {'Web Development':0.0,
 'Business Finance':1.0,
 'Musical Instruments':2.0,
 'Graphic Design':3.0}

In [21]:
df = labelEncoder.transform(df)

In [22]:
(trainDF,testDF) = df.randomSplit((0.7,0.3),seed=42)

In [23]:
from pyspark.ml.classification import LogisticRegression

In [24]:
lr = LogisticRegression(featuresCol='vectorizedFeatures',labelCol='label')

In [25]:
from pyspark.ml import Pipeline 

In [26]:
pipeline = Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,lr])

In [27]:
lr_model = pipeline.fit(trainDF)

In [28]:
predictions = lr_model.transform(testDF)

In [29]:
predictions.show()

+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               title|            category|label|            mytokens|     filtered_tokens|         rawFeatures|  vectorizedFeatures|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| 18 Ounce Stainle...|      Home & Kitchen|  1.0|[, 18, ounce, sta...|[, 18, ounce, sta...|(36504,[2,10,18,3...|(36504,[2,10,18,3...|[-2.7293188370913...|[1.53235742404087...|       1.0|
| A=1/2-13, B=3/4,...|                None|  0.0|[, a=1/2-13,, b=3...|[, a=1/2-13,, b=3...|(36504,[38,67,310...|(36504,[38,67,310...|[-42.745313844737...|[2.29466142985496...|       2.0|
| Aluminum Stub Pi...|                None|  0.0|[, aluminum, stu

In [30]:
predictions.columns

['title',
 'category',
 'label',
 'mytokens',
 'filtered_tokens',
 'rawFeatures',
 'vectorizedFeatures',
 'rawPrediction',
 'probability',
 'prediction']

In [31]:
predictions.select('rawPrediction','probability','category','label','prediction').show(10)

+--------------------+--------------------+--------------------+-----+----------+
|       rawPrediction|         probability|            category|label|prediction|
+--------------------+--------------------+--------------------+-----+----------+
|[-2.7293188370913...|[1.53235742404087...|      Home & Kitchen|  1.0|       1.0|
|[-42.745313844737...|[2.29466142985496...|                None|  0.0|       2.0|
|[28.9562091527174...|[0.11169076173533...|                None|  0.0|       2.0|
|[3.57732605322395...|[9.15925989534345...|                None|  0.0|       1.0|
|[105.669743011312...|[1.0,2.0595660966...|      Home & Kitchen|  1.0|       0.0|
|[44.2813359096834...|[0.99999999999994...|     Office Products|  3.0|       0.0|
|[159.323812020586...|[1.0,8.1379190830...|      Home & Kitchen|  1.0|       0.0|
|[57.3473370567197...|[1.0,8.5226369429...|Grocery & Gourmet...|  4.0|       0.0|
|[2.16209902796626...|[1.37387079672839...|      Home & Kitchen|  1.0|       1.0|
|[34.37058543377

In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [33]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')

In [34]:
accuracy = evaluator.evaluate(predictions)

In [35]:
accuracy

0.42385321100917434

# Fine tuning

In [36]:
# import pyspark.ml.tuning as tune

# # Create the parameter grid
# grid = tune.ParamGridBuilder()

# # Add the hyperparameter
# grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))
# grid = grid.addGrid(lr.elasticNetParam, [0, 1])

# # Build the grid
# grid = grid.build()

In [43]:
df.columns

['title', 'category', 'label']

In [37]:
# Create the CrossValidator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
# cv = tune.CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator)

In [45]:


#training_data = trainDF
#testing_data = testDF

paramGrid = ParamGridBuilder().addGrid(lr.regParam,[0.02,0.08])\
            .addGrid(lr.elasticNetParam,[0.2,0.6]).build()

#evaluator_lr = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
evaluator_lr = evaluator
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator_lr,
                          numFolds=2)

#start_time = time()
pipeline2 = Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf])
piped_data = pipeline2.fit(df).transform(df)

# Split the data into training and test sets
training, test = piped_data.randomSplit([.7, .3])

#best_lr = lr.fit(training)

# # Print best_lr
#print(best_lr)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)

#end_time = time()

#training_time = end_time - start_time

#print("The time taken to train the data is: %0.3f seconds" %training_time)

# Make predictions on testing data and calculating ROC metrics and model accuracy. 
prediction = cvModel.transform(test)
# #output= prediction.select("features",  "probability", "prediction")
# output = prediction
# acc = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "accuracy"})
# f1 = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "f1"})
# weightedPrecision = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "weightedPrecision"})
# weightedRecall = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "weightedRecall"})
# auc = evaluator_lr.evaluate(output)

# print(acc)
# print(f1)
# print(weightedPrecision)
# print(weightedRecall)
# print(auc)

AnalysisException: cannot resolve 'features' given input columns: [category, filtered_tokens, label, mytokens, prediction, probability, rawFeatures, rawPrediction, title, vectorizedFeatures];
'Project ['features, probability#8259, prediction#8275]
+- Project [title#16, category#17, label#86, mytokens#1651, filtered_tokens#1662, rawFeatures#1674, vectorizedFeatures#1687, rawPrediction#8247, probability#8259, UDF(rawPrediction#8247) AS prediction#8275]
   +- Project [title#16, category#17, label#86, mytokens#1651, filtered_tokens#1662, rawFeatures#1674, vectorizedFeatures#1687, rawPrediction#8247, UDF(rawPrediction#8247) AS probability#8259]
      +- Project [title#16, category#17, label#86, mytokens#1651, filtered_tokens#1662, rawFeatures#1674, vectorizedFeatures#1687, UDF(vectorizedFeatures#1687) AS rawPrediction#8247]
         +- Sample 0.7, 1.0, false, 5899551277767315362
            +- Sort [title#16 ASC NULLS FIRST, category#17 ASC NULLS FIRST, label#86 ASC NULLS FIRST, mytokens#16

In [46]:
output = prediction
acc = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "accuracy"})
f1 = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "f1"})
weightedPrecision = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "weightedPrecision"})
weightedRecall = evaluator_lr.evaluate(output, {evaluator_lr.metricName: "weightedRecall"})
auc = evaluator_lr.evaluate(output)

print(acc)
print(f1)
print(weightedPrecision)
print(weightedRecall)
print(auc)

0.49357827995447895
0.3313665488704755
0.4444602567588213
0.49357827995447895
0.49357827995447895


# Step 3.

In [36]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, StringType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [37]:
data=df
train, validation, test = data.randomSplit([0.7, 0.2, 0.1], 1234)

In [38]:
categorical_columns = [item[0] for item in data.dtypes if item[1].startswith(
    'string')]
numeric_columns = [item[0] for item in data.dtypes if item[1].startswith(
    'double')]
indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(
    column)) for column in categorical_columns]

In [51]:
data.dtypes

[('title', 'string'), ('category', 'string'), ('label', 'double')]

In [53]:
[indexer.getOutputCol() for indexer in indexers] 

['title_index', 'category_index']

In [43]:
featuresCreator = VectorAssembler(
    inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns,
    outputCol='features')
layers = [len(featuresCreator.getInputCols()), 4, 2, 12]

In [44]:
print(layers)

[3, 4, 2, 12]


In [45]:
classifier = MultilayerPerceptronClassifier(labelCol='label',
                                            featuresCol='features',
                                            maxIter=100,
                                            layers=layers,
                                            blockSize=128,
                                            seed=1234)

In [46]:
pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])
model = pipeline.fit(train)


In [47]:
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)

In [48]:
train_predictionAndLabels = train_output_df.select('prediction', 'label')
validation_predictionAndLabels = validation_output_df.select('prediction', 'label')
test_predictionAndLabels = test_output_df.select('prediction', 'label')

In [50]:
metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']
for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(metricName=metric)
    print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
    print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
    print('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))

Train accuracy = 0.5004948395306094


Py4JJavaError: An error occurred while calling o1066.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 192.0 failed 1 times, most recent failure: Lost task 0.0 in stage 192.0 (TID 185) (host.docker.internal executor driver): org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$3277/0x0000000101408840: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at scala.coll